In [ ]:
# Importing Libraries
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import scipy as sp
import numpy as np
import pandas as pd
import time
import datetime
import os
import sys
import pyvisa
import threading
import ipywidgets
from IPython.display import display

print(f"All Libraries imported successfully at {datetime.datetime.now()}")

In [ ]:
# Scan Connected instruments Instruments
def scan_instrument():
    rm = pyvisa.ResourceManager() # Open the Resource Manager
    instruments = []
    for address in rm.list_resources(): # Running over all the available addresses in the Resource Manager
        try:
            instrument = rm.open_resource(address)
            responce = instrument.query('*IDN?').strip()
            instruments.append(f"{address:<50} => {responce}")
        except Exception:
            responce = "No responce"
            instruments.append(f"{address:<50} => {responce}")

    # AMI430 might not respond to *IDN? query, so we handle it separately
    address = 'TCPIP::192.168.1.30::7180::SOCKET'
    try:
        instrument = rm.open_resource(address, timeout=5000, write_termination='\n', read_termination='\n')
        responce = instrument.query("*IDN?")
        instruments.append(f"{address:<50} => {responce}")
    except Exception:
        responce = "No responce"
        instruments.append(f"{address:<50} => {responce}")
    return instruments

for items in scan_instrument():
    print(items)

print(f"\nInstruments scanned successfully at {datetime.datetime.now()}")

In [ ]:
# connecting to the instrument
rm = pyvisa.ResourceManager()
# SR860 = rm.open_resource("TCPIP0::192.168.1.20::inst0::INSTR")
# SR830 = rm.open_resource("GPIB1::6::INSTR")
# SR830_1 = rm.open_resource("GPIB1::7::INSTR")
# SR830_2 = rm.open_resource("GPIB1::8::INSTR")
# lakeshore = rm.open_resource("GPIB1::5::INSTR")
# ami430 = rm.open_resource('TCPIP::192.168.1.30::7180::SOCKET', timeout=5000, write_termination='\n', read_termination='\n')

print(f"Instruments connected successfully at {datetime.datetime.now()}")

In [ ]:
# Helper functions

# Function for defining the stop button click event handler
def on_stop_clicked(b):
    stop_event.set()
    status_btn.description = "Stopped"
    status_btn.button_style = "danger"
    stop_btn.disabled = True
    stop_btn.close()

# Function for creating path for saving file in a specific folder
def save_file(folder,filename):
    current_path = os.getcwd()
    folder_path = os.path.join(current_path, folder)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    filename = f"{datetime.datetime.now().strftime('%y%m%d')}_{filename}"
    file_path = os.path.join(folder_path, filename)
    return file_path

# Function for creating path for loading file from a specific folder
def load_file(folder,filename):
    current_path = os.getcwd()
    filename = f"{datetime.datetime.now().strftime('%y%m%d')}_{filename}"
    file_path = os.path.join(current_path, folder, filename)
    return file_path

# Function to measure X, Y, R, T from SR830
def measure_SR830(address):
    X_comp=address.query_ascii_values('OUTP? 1')[0]
    Y_comp=address.query_ascii_values('OUTP? 2')[0]
    R_comp=address.query_ascii_values('OUTP? 3')[0]
    T_comp=address.query_ascii_values('OUTP? 4')[0]
    return X_comp,Y_comp,R_comp,T_comp

# Function to measure X, Y, R, T from SR860
def measure_SR860(address):
    X_comp=address.query_ascii_values('OUTP? 0')[0]
    Y_comp=address.query_ascii_values('OUTP? 1')[0]
    R_comp=address.query_ascii_values('OUTP? 2')[0]
    T_comp=address.query_ascii_values('OUTP? 3')[0]
    return X_comp,Y_comp,R_comp,T_comp

# Function for converting XYRT to dVdI
def calculate_dVdI(X,Y,R,I,Gain):
    dVdI_x=X/(I*Gain)
    dVdI_y=Y/(I*Gain)
    dVdI_r=R/(I*Gain)
    return dVdI_x,dVdI_y,dVdI_r

# Function for sweeping frequency in Lock-in Amplifier SR830 and SR860
def frequency_ramp(fset,fsubstep,step_time,instrument,address):
    if instrument == 'SR860':
        if fset < 1e-3:
            fset = 1e-3
        elif fset > 500e3:
            print('Frequency setpoint should be between 1 mHz to 500 kHz')
            sys.exit()
        fnow=address.query_ascii_values('FREQ?')[0] # reading current value
        N=abs(fset-fnow)/fsubstep # dividing the total voltage range into small steps
        for i in np.linspace(fnow,fset,int(N)):
            address.write(f'FREQ {i}')
            time.sleep(step_time)
    elif instrument == 'SR830':
        if fset < 1e-3:
            fset = 1e-3
        elif fset > 102e3:
            print('Frequency setpoint should be between 1 mHz to 102 kHz')
            sys.exit()
        fnow=address.query_ascii_values('FREQ?')[0] # reading current value
        N=abs(fset-fnow)/fsubstep # dividing the total voltage range into small steps
        for i in np.linspace(fnow,fset,int(N)):
            address.write(f'FREQ {i}')
            time.sleep(step_time)
    else:
        print('Error: Invalid Instrument')
        sys.exit()

# Function for ramping vsine in small steps for SR830 and SR860
def vsine_ramp(vset,vsubstep,step_time,instrument,address):
    if instrument == 'SR830':
        if vset < 0.004:
            vset = 0.004
        elif vset > 5:
            print('Voltage setpoint should be between 4mV to 5 V')
            sys.exit()
        vnow=address.query_ascii_values('SLVL?')[0] # reading current value
        N=abs(vset-vnow)/vsubstep # dividing the total voltage range into small steps
        for i in np.linspace(vnow,vset,int(N)):
            address.write(f'SLVL {i}')
            time.sleep(step_time)
    elif instrument == 'SR860':
        if vset < 1e-9:
            vset = 1e-9
        elif vset > 2:
            print('Voltage setpoint should be between 1 nV to 2 V')
            sys.exit()
        vnow=address.query_ascii_values('SLVL?')[0] # reading current value
        N=abs(vset-vnow)/vsubstep # dividing the total voltage range into small steps
        for i in np.linspace(vnow,vset,int(N)):
            address.write(f'SLVL {i}')
            time.sleep(step_time)
    else:
        print('Error: Invalid Instrument')
        sys.exit()

# Function for ramping vsine to zero for SR830 and SR860
def vsine_ramp_zero(vsubstep,step_time,instrument,address):
    vsine_ramp(0,vsubstep,step_time,instrument,address)

# Function for ramping dc voltage in small steps
# Instrument: SMU2450, SMU2400, EMU6517A, WFG33120A, WFG33500B, SR830AUX1, SR830AUX2, SR830AUX3, SR830AUX4
def vdc_ramp(vset,vsubstep,step_time,instrument,address):
    if instrument == 'SMU2450': # Keithley 2450 Sourcemeter Unit
        address.write('*WAI') # 2450 wait for the previous command to finish
        vnow=address.query_ascii_values(':SOUR:VOLT?')[0] # reading current value
        N=abs(vset-vnow)/vsubstep # dividing the total voltage range into small steps
        for i in np.linspace(vnow,vset,int(N)):
            address.write(f':SOUR:VOLT {i}')
            time.sleep(step_time)
    elif instrument == 'SMU2400': # Keithley 2400 Sourcemeter Unit
        address.write('*WAI') # 2400 wait for the previous command to finish
        vnow=address.query_ascii_values(':SOUR:VOLT?')[0] # reading current value
        N=abs(vset-vnow)/vsubstep # dividing the total voltage range into small steps
        for i in np.linspace(vnow,vset,int(N)):
            address.write(f':SOUR:VOLT {i}')
            time.sleep(step_time)
    elif instrument == 'B2910BL': # Keithley 2400 Sourcemeter Unit
        address.write('*WAI') # 2400 wait for the previous command to finish
        vnow=float(address.query(':SOUR:VOLT?')) # reading current value
        N=abs(vset-vnow)/vsubstep # dividing the total voltage range into small steps
        for i in np.linspace(vnow,vset,int(N)):
            address.write(f':SOUR:VOLT {i}')
            time.sleep(step_time)
    elif instrument == 'EMU6517A': # Keithley 6517A Eletrometer Unit
        vnow=address.query_ascii_values(':SOUR:VOLT?')[0] # reading current value
        N=abs(vset-vnow)/vsubstep # dividing the total voltage range into small steps
        for i in np.linspace(vnow,vset,int(N)):
            address.write(f':SOUR:VOLT {i}')
            time.sleep(step_time)
    elif instrument == 'WFG33120A': # Agilent/HP 33120A Waveform Generator
        vnow=address.query_ascii_values('VOLT:OFFS?')[0] # reading current value
        N=abs(vset-vnow)/vsubstep # dividing the total voltage range into small steps
        for i in np.linspace(vnow,vset,int(N)):
            address.write('APPLy:DC DEF, DEF, ',str(i))
            time.sleep(step_time)
    elif instrument == 'WFG33500B': # Keysight 33500B Waveform Generator
        vnow=address.query_ascii_values('VOLT:OFFS?')[0] # reading current value
        N=abs(vset-vnow)/vsubstep # dividing the total voltage range into small steps
        for i in np.linspace(vnow,vset,int(N)):
            address.write('APPLy:DC DEF, DEF, ',str(i))
            time.sleep(step_time)
    elif instrument == 'SR830AUX1': # SRS SR830 Lock-in Amplifier AUX1
        vnow=address.query_ascii_values('AUXV? 1')[0] # reading current value
        N=abs(vset-vnow)/vsubstep # dividing the total voltage range into small steps
        for i in np.linspace(vnow,vset,int(N)):
            address.write('AUXV1,',str(i))
            time.sleep(step_time)
    elif instrument == 'SR830AUX2': # SRS SR830 Lock-in Amplifier AUX2
        vnow=address.query_ascii_values('AUXV? 2')[0] # reading current value
        N=abs(vset-vnow)/vsubstep # dividing the total voltage range into small steps
        for i in np.linspace(vnow,vset,int(N)):
            address.write('AUXV2,',str(i))
            time.sleep(step_time)
    elif instrument == 'SR830AUX3': # SRS SR830 Lock-in Amplifier AUX3
        vnow=address.query_ascii_values('AUXV? 3')[0] # reading current value
        N=abs(vset-vnow)/vsubstep # dividing the total voltage range into small steps
        for i in np.linspace(vnow,vset,int(N)):
            address.write('AUXV3,',str(i))
            time.sleep(step_time)
    elif instrument == 'SR830AUX4': # SRS SR830 Lock-in Amplifier AUX4
        vnow=address.query_ascii_values('AUXV? 4')[0] # reading current value
        N=abs(vset-vnow)/vsubstep # dividing the total voltage range into small steps
        for i in np.linspace(vnow,vset,int(N)):
            address.write('AUXV4,',str(i))
            time.sleep(step_time)
    elif instrument == 'SR860': # SRS SR830 Lock-in Amplifier combined
        vnow=address.query_ascii_values('SOFF?')[0] # reading current value
        N=abs(vset-vnow)/vsubstep # dividing the total voltage range into small steps
        for i in np.linspace(vnow,vset,int(N)):
            address.write('SOFF ',str(i))
            time.sleep(step_time)   
    else:
        print('Error: Invalid Instrument')
        sys.exit()

# Function for ramping dc voltage to zero
def vdc_ramp_zero(rstep,rtime,instrument,address):
    vdc_ramp(0,rstep,rtime,instrument,address)

# Function for creating array for sweeping
def sweep(vmin, vmax, vstep, vmode): # Modes: up, down, updown, downup, round-up, round-down
    if vmode =='up': # lower to higher value
        sweep = np.arange(vmin,vmax+vstep,vstep)
    elif vmode =='down': # higher to lower value
        sweep = np.arange(vmax,vmin-vstep,-vstep)
    elif vmode =='updown': # lower to higher to lower value
        sweep = np.arange(vmin,vmax+vstep,vstep)
        sweep = np.append(sweep,np.arange(vmax-vstep,vmin-vstep,-vstep))
    elif vmode =='downup': # higher to lower to higher value
        sweep = np.arange(vmax,vmin-vstep,-vstep)
        sweep = np.append(sweep,np.arange(vmin+vstep,vmax+vstep,vstep))
    elif vmode =='round-up': # zero to higher to lower to zero value
        sweep = np.arange(0,vmax+vstep,vstep)
        sweep = np.append(sweep,np.arange(vmax-vstep,vmin-vstep,-vstep))
        sweep = np.append(sweep,np.arange(vmin+vstep,vstep,vstep))
    elif vmode =='round-down': # zero to lower to higher to zero value
        sweep = np.arange(0,vmin-vstep,-vstep)
        sweep = np.append(sweep,np.arange(vmin+vstep,vmax+vstep,vstep))
        sweep = np.append(sweep,np.arange(vmax-vstep,-vstep,-vstep))
    else:
        print('Error: Invalid Sweep Mode')
        sys.exit()
    return sweep

# Function for converting current values to voltage values
def ItoV(i_min,i_max,i_step,R_series):
    '''returns v_min,v_max,v_step'''
    if R_series == 0:
        R_series = 1
    v_min=i_min*R_series
    v_max=i_max*R_series
    v_step=i_step*R_series
    return v_min,v_max,v_step

# Function for converting voltage values to current values
def VtoI(v_min,v_max,v_step,R_series):
    '''returns i_min,i_max,i_step'''
    if R_series == 0:
        R_series = 1
    i_min=v_min/R_series
    i_max=v_max/R_series
    i_step=v_step/R_series
    return i_min,i_max,i_step

# Function to read temperature from lakeshore via gpib
def readtemp_lakeshore(connection,address, channel):
    if connection == 'GPIB':
        try:
            temp = address.query_ascii_values(f"RDGK? {channel}")[0]
        except Exception:
            temp = np.nan
    return temp

# Functions to set heater parameters
def set_heater(connection,address,mode, range, P, I, D):
    '''
    Set heater parameters:
    mode: Heater mode - "Closed Loop PID","Zone Tuning","Open Loop","Off"
    range: Heater range - "off","31.6 uA","100 uA","316 uA","1.00 mA","3.16 mA","10.0 mA","31.6 mA","100 mA"
    P, I, D: PID parameters (only for Closed Loop PID mode)
    '''
    if connection == 'GPIB':
        heater_range_list = ["off","31.6 uA","100 uA","316 uA","1.00 mA","3.16 mA","10.0 mA","31.6 mA","100 mA"]
        heater_mode_list = ["Closed Loop PID","Zone Tuning","Open Loop","Off"]
        if mode in heater_mode_list:
            mode_index = heater_mode_list.index(mode) + 1
            address.write(f'CMODE {mode_index}')
        else:
            print('Error: Invalid Heater Mode')
            sys.exit()
        if mode == "Closed Loop PID":
            if range in heater_range_list:
                range_index = heater_range_list.index(range)
                address.write(f'HTRRNG {range_index}')
            else:
                print('Error: Invalid Heater Range')
                sys.exit()
            address.write(f'PID {P},{I},{D}')
        print('Heater settings updated successfully')

# Function to get heater status        
def get_heater_status(connection,address):
    if connection == 'GPIB':
        # Function to read heater mode
        def read_heater_mode(address):
            n = address.query_ascii_values('CMODE?')[0]
            heater_mode_list = ["Closed Loop PID","Zone Tuning","Open Loop","Off"]
            val = heater_mode_list[int(n)-1]
            return val
        # Function to read heater range
        def read_heater_range(address):
            n = address.query_ascii_values('HTRRNG?')[0]
            heater_range_list = ["off","31.6 uA","100 uA","316 uA","1.00 mA","3.16 mA","10.0 mA","31.6 mA","100 mA"]
            val = heater_range_list[int(n)]
            return val
        # Function to read PID parameters
        def read_pid_params(address):
            params = address.query_ascii_values('PID?')
            return params
        # Reading all heater status
        mode = read_heater_mode(address)
        range = read_heater_range(address)
        P, I, D = read_pid_params(address)
        status = {
            "Mode": mode,
            "Range": range,
            "P": P,
            "I": I,
            "D": D
        }
    return status

# Function to set AMI430 field and ramp rate units
def set_ami_unit(field_unit, rate_unit, ami430):
    '''
    Tesla or kiloGauss for field_unit. per_minute or per_second for rate_unit.
    1 Tesla = 10 kiloGauss
    '''
    try:
        if field_unit == 'Tesla':
            field_unit_code = 1
        elif field_unit == 'kiloGauss':
            field_unit_code = 0
        if rate_unit == 'per_minute':
            rate_unit_code = 1
        elif rate_unit == 'per_second':
            rate_unit_code = 0
        ami430.write(f"CONFigure:FIELD:UNITS {field_unit_code}")
        ami430.write(f"CONFigure:RAMP:RATE:UNITS {rate_unit_code}")
        print(f'Units set to {field_unit} {rate_unit}')
    except:
        print('Error: Unable to set units. Check AMI430 and retry.')
        sys.exit()

print(f"All Helper functions are defined successfully at {datetime.datetime.now()}")

In [ ]:
# Program for recording R vs Time from 1 SR830 Lock-in Amplifier (1D Sweep)

# Status and Stop Buttons
stop_event = threading.Event()
status_btn = ipywidgets.Button(description="Running", button_style="success", disabled=True)
stop_btn = ipywidgets.Button(description="Stop measurement", button_style="danger")
display(status_btn,stop_btn)
stop_btn.on_click(on_stop_clicked)
# Measurement Loop in a separate thread
def measurement_loop():
    filename = save_file("Date", "measurement_file_name.dat")
    if not os.path.exists(filename):
        cols = ["datetime", "Vin", "Vx", "Vy", "Iin", "Rx", "Ry"]
        pd.DataFrame(columns=cols).to_csv(filename, index=False)
    
    R_series = 1e6 # Series resistance in Ohms
    Gain = 1    # Gain of the pre-amplifier

    while not stop_event.is_set():
        t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        Vin = SR830_1.query_ascii_values('SLVL?')[0]
        Iin = Vin / R_series
        Vx, Vy, Vr, Phase = measure_SR830(SR830_1)
        Rx = Vx / (Iin * Gain)
        Ry = Vy / (Iin * Gain)
        Rr = Vr / (Iin * Gain)

        row = {"datetime": t, "Vin": Vin, "Vx": Vx, "Vy": Vy,"Vr": Vr,"Phase": Phase, "Iin": Iin, "Rx": Rx, "Ry": Ry, "Rr": Rr}
        pd.DataFrame([row]).to_csv(filename, mode='a', header=False, index=False)
        time.sleep(1)

threading.Thread(target=measurement_loop, daemon=True).start()

In [ ]:
# Program for recording R vs Time from 2 SR830 Lock-in Amplifiers (1D Sweep)

# Status and Stop Buttons
stop_event = threading.Event()
status_btn = ipywidgets.Button(description="Running", button_style="success", disabled=True)
stop_btn = ipywidgets.Button(description="Stop measurement", button_style="danger")
display(status_btn,stop_btn)
stop_btn.on_click(on_stop_clicked)

# Measurement Loop in a separate thread
def measurement_loop():
    filename = save_file("Date", "measurement_file_name.dat")
    if not os.path.exists(filename):
        cols = ["datetime", "Vin", "Vx", "Vy", "Iin", "Rx", "Ry"]
        pd.DataFrame(columns=cols).to_csv(filename, index=False)
    
    R_series_1 = 1e6 # Series resistance in Ohms
    R_series_2 = 1e6 # Series resistance in Ohms
    Gain_1 = 1    # Gain of the pre-amplifier
    Gain_2 = 1    # Gain of the pre-amplifier

    while not stop_event.is_set():
        t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        Vin1 = SR830_1.query_ascii_values('SLVL?')[0]
        Iin1 = Vin1 / R_series_1
        Vx1, Vy1, Vr1, Phase1 = measure_SR830(SR830_1)
        Rx1 = Vx1 / (Iin1 * Gain_1)
        Ry1 = Vy1 / (Iin1 * Gain_1)
        Rr1 = Vr1 / (Iin1 * Gain_1)

        Vin2 = SR830_2.query_ascii_values('SLVL?')[0]
        Iin2 = Vin2 / R_series_2
        Vx2, Vy2, Vr2, Phase2 = measure_SR830(SR830_2)
        Rx2 = Vx2 / (Iin2 * Gain_2)
        Ry2 = Vy2 / (Iin2 * Gain_2)
        Rr2 = Vr2 / (Iin2 * Gain_2)

        row = {"datetime": t, 
                "Vin1": Vin1, "Vx1": Vx1, "Vy1": Vy1,"Vr1": Vr1,"Phase1": Phase1, "Iin1": Iin1, "Rx1": Rx1, "Ry1": Ry1, "Rr1": Rr1,
                "Vin2": Vin2, "Vx2": Vx2, "Vy2": Vy2, "Vr2": Vr2,"Phase2": Phase2, "Iin2": Iin2, "Rx2": Rx2, "Ry2": Ry2, "Rr2": Rr2}
        pd.DataFrame([row]).to_csv(filename, mode='a', header=False, index=False)
        time.sleep(1)

threading.Thread(target=measurement_loop, daemon=True).start()

In [ ]:
# Program for recording dV/dI values from 1 SR860 Lock-in Amplifier (1D Sweep)

# Status and Stop Buttons
stop_event = threading.Event()
status_btn = ipywidgets.Button(description="Running", button_style="success", disabled=True)
stop_btn = ipywidgets.Button(description="Stop measurement", button_style="danger")
display(status_btn, stop_btn)
stop_btn.on_click(on_stop_clicked)

# Measurement Loop in a separate thread
def measurement_loop():
    filename = save_file("Date", "measurement_file_name.dat")
    if not os.path.exists(filename):
        cols = ["datetime", "Vin", "Vx", "Vy", "Iin", "dVdIx", "dVdIy"]
        pd.DataFrame(columns=cols).to_csv(filename, index=False)
    
    Rseries = 1e6  # Series resistance in Ohms
    Gain = 1    # Gain setting of the current amplifier
    Iac = 0.1   # AC current applied in Amperes
    Idc_min = -0.001  # Minimum DC current in Amperes
    Idc_max = 0.001   # Maximum DC current in Amperes
    Idc_step = 0.0001  # DC current step in Amperes

    Vac = Iac * Rseries  # AC voltage applied in Volts
    vsine_ramp(Vac,0.0001,0.05,'SR860',SR860)  # Ramping vsine to desired value
    Vdc_min, Vdc_max, Vdc_step = ItoV(Idc_min, Idc_max, Idc_step, Rseries) # Converting current values to voltage values

    for Vbias in sweep(Vdc_min,Vdc_max,Vdc_step,'up'):
        t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        vdc_ramp(Vbias,vbias_step,0.1,'SR860',SR860) # Ramping vdc to desired value
        time.sleep(1)
        Idc = Vbias / Rseries
        Vx, Vy, Vr, Phase = measure_SR860(SR860)
        dVdIx, dVdIy, dVdIr = calculate_dVdI(Vx, Vy, Vr, Iac, Gain)

        row = {"datetime": t, "Vac": Vac, "Vdc": Vdc, "Vx": Vx, "Vy": Vy, "Vr": Vr, "Phase": Phase, 
                "Iac": Iac, "Idc": Idc, "dVdIx": dVdIx, "dVdIy": dVdIy, "dVdIr": dVdIr}
        pd.DataFrame([row]).to_csv(filename, mode='a', header=False, index=False)

        if stop_event.is_set():
            break

    vdc_ramp(0,0.001,0.01,'SR860',SR860)
    vsine_ramp(0,0.0001,0.1,'SR860',SR860)
    
    # Finalize UI state
    if stop_event.is_set():
        status_btn.description = "Stopped"
        status_btn.button_style = "danger"
    else:
        status_btn.description = "Completed"
        status_btn.button_style = "info"
    stop_btn.disabled = True    
    stop_event.set()
    stop_btn.disabled = True
    stop_btn.close()
    stop_event.set()

threading.Thread(target=measurement_loop, daemon=True).start()

In [ ]:
# Custom Program Template (2D Sweep)

# Status and Stop Buttons
stop_event = threading.Event()
status_btn = ipywidgets.Button(description="Running", button_style="success", disabled=True)
stop_btn = ipywidgets.Button(description="Stop measurement", button_style="danger")
display(status_btn, stop_btn)
stop_btn.on_click(on_stop_clicked)

# Measurement Loop in a separate thread
def measurement_loop():
    '''
    filesaving codes and measurement parameters code will go here. 
    
    '''
    for i in sweep(0,10,1,'up'): # your loop code will go here
        '''
        your measurement code will go here.
        You can save data to file inside this loop as shown in previous examples.
        '''
        for j in sweep(11,20,1,'down'):
            '''
            your nested measurement code will go here.
            You can save data to file inside this loop as shown in previous examples.
            '''
            if stop_event.is_set():
                break
    
    '''
    your ramp down code will go here.
    '''
    # Finalize UI state
    if stop_event.is_set():
        status_btn.description = "Stopped"
        status_btn.button_style = "danger"
    else:
        status_btn.description = "Completed"
        status_btn.button_style = "info"
    stop_btn.disabled = True    
    stop_event.set()
    stop_btn.disabled = True
    stop_btn.close()
    stop_event.set()

threading.Thread(target=measurement_loop, daemon=True).start()

In [ ]:
# Custom programs template (1D Sweep)

# Status and Stop Buttons
stop_event = threading.Event()
status_btn = ipywidgets.Button(description="Running", button_style="success", disabled=True)
stop_btn = ipywidgets.Button(description="Stop measurement", button_style="danger")
display(status_btn, stop_btn)
stop_btn.on_click(on_stop_clicked)

# Measurement Loop in a separate thread
def measurement_loop():
    '''
    filesaving codes and measurement parameters code will go here. 
    
    '''
    for i in sweep(0,10,1,'up'): # your loop code will go here
        '''
        your measurement code will go here.
        You can save data to file inside this loop as shown in previous examples.
        '''
        if stop_event.is_set():
            break
    
    '''
    your ramp down code will go here.
    '''
    # Finalize UI state
    if stop_event.is_set():
        status_btn.description = "Stopped"
        status_btn.button_style = "danger"
    else:
        status_btn.description = "Completed"
        status_btn.button_style = "info"
    stop_btn.disabled = True    
    stop_event.set()
    stop_btn.disabled = True
    stop_btn.close()
    stop_event.set()

threading.Thread(target=measurement_loop, daemon=True).start()

In [ ]:
# Program for changing and logging magnetic field using AMI430

B_min = 0  # Minimum magnetic field in Tesla
B_max = 1  # Maximum magnetic field in Tesla
B_step = 0.1  # Magnetic field step in Tesla
B_ramp_rate = 0.002  # Ramp rate in Tesla per minute

# Configuring AMI430
set_ami_unit('Tesla', 'per_minute', ami430)

ami430.write('CONFigure:DEV:MODE:')

# Open log file and write header
logfile = save_file("Magnet Log", "measurement_file_name.dat")
if not os.path.exists(logfile):
    cols = ["datetime", "Temperature", "Field", "Vmagnet", "Vsupply"]
    pd.DataFrame(columns=cols).to_csv(logfile, index=False)

for B in sweep(B_min, B_max, B_step, 'up'):
    t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    Temp = readtemp_lakeshore_gpib(lakeshore, 3)
    Field = ami430.query_ascii_values('FIELD:MAGnet?')[0]
    V_magnet = ami430.query_ascii_values('VOLTage:MAGnet?')[0]
    V_supply = ami430.query_ascii_values('VOLTage:SUPPly?')[0]

    row = {"datetime": t, "Temperature": Temp, "Field": Field, "Vmagnet": V_magnet, "Vsupply": V_supply}
    pd.DataFrame([row]).to_csv(logfile, mode='a', header=False, index=False)

In [ ]:
def get_ami_status(ami430):
    ami_state_number = ami430.query_ascii_values('STATE?')
    ami_state_list = ["RAMPING to target", 
                      "HOLDING at target", 
                      "PAUSED", 
                      "Ramping in MANUAL UP mode", 
                      "Ramping in MANUAL DOWN mode", 
                      "ZEROING CURRENT", 
                      "QUENCH DETECTED", 
                      "AT ZERO current", 
                      "Heating persistent switch", 
                      "Cooling persistent switch"]
    ami_state = ami_state_list[int(ami_state_number[0])-1]
    print(f"AMI430 Status: {ami_state}", flush=True, end='\r')
    return ami_state_number

# Function to set AMI430 magnetic field
def set_ami_field(B_set, ami430):
    ami_state_number = ami430.query_ascii_values('STATE?')[0]
    if ami_state_number in [1, 2, 3]:
        ami430.write('CONFigure:RAMP:RATE:SEGment 1')
        ami430.write(f'CONFigure:FIELD:TARget {B_set}')
        ami430.write(f'RAMP')

def set_ami_ramp_rate(rate, field_limit, ami430):
    ami430.write('CONFigure:RAMP:RATE:SEGment 1')
    ami430.write(f'CONFigure:RAMP:RATE:FIELD 1,{rate},{field_limit}')